In [1]:
import pandas as pd
from datetime import datetime
import os

class PortfolioTracker:
    """投资组合跟踪器"""
    
    def __init__(self, filename='portfolio_tracker.xlsx'):
        self.filename = filename
        self.transactions_sheet = '交易记录'
        self.holdings_sheet = '当前持仓'
        
    def initialize_portfolio(self, initial_cash=0, existing_holdings=None):
        """
        初始化投资组合
        
        参数:
            initial_cash: 初始现金金额
            existing_holdings: 现有持仓列表,支持两种格式:
                方式1(推荐): 使用当前净值和盈亏比例
                [
                    {
                        '资产名称': '华泰柏瑞红利ETF', 
                        '持有数量': 16506.42, 
                        '当前净值': 1.0951,  # 当前价格/净值
                        '盈亏比例': -2.5  # 当前盈亏百分比,如+5.5%或-3.2%
                    },
                ]
                方式2: 直接提供平均成本
                [
                    {
                        '资产名称': '贵州茅台', 
                        '持有数量': 100, 
                        '平均成本': 1650.00,
                        '盈亏比例': 5.5  # 可选
                    },
                ]
        """
        # 创建交易记录表
        transactions_df = pd.DataFrame(columns=[
            '日期', '操作类型', '资产名称', '数量', '价格', '金额', '手续费', '实际成本', '备注'
        ])
        
        current_date = datetime.now().strftime('%Y-%m-%d')
        
        # 记录初始现金
        if initial_cash > 0:
            cash_record = pd.DataFrame([{
                '日期': current_date,
                '操作类型': '初始化',
                '资产名称': '现金',
                '数量': 1,
                '价格': initial_cash,
                '金额': initial_cash,
                '手续费': 0,
                '实际成本': initial_cash,
                '备注': '初始现金'
            }])
            transactions_df = pd.concat([transactions_df, cash_record], ignore_index=True)
        
        # 记录现有持仓
        if existing_holdings:
            for holding in existing_holdings:
                asset_name = holding['资产名称']
                quantity = holding['持有数量']
                profit_pct = holding.get('盈亏比例', 0)
                
                # 判断使用哪种方式,并计算相应的值
                if '当前净值' in holding:
                    # 方式1: 使用当前净值和盈亏比例反推
                    current_price = holding['当前净值']
                    current_value = quantity * current_price
                    
                    if profit_pct != 0:
                        total_cost = current_value / (1 + profit_pct / 100)
                    else:
                        total_cost = current_value
                    
                    avg_cost = total_cost / quantity
                    
                elif '平均成本' in holding:
                    # 方式2: 使用平均成本
                    avg_cost = holding['平均成本']
                    total_cost = quantity * avg_cost
                    current_price = avg_cost
                else:
                    raise ValueError(f"资产 {asset_name} 必须提供 '当前净值' 或 '平均成本'")
                
                # 记录到交易表
                holding_record = pd.DataFrame([{
                    '日期': current_date,
                    '操作类型': '初始化',
                    '资产名称': asset_name,
                    '数量': quantity,
                    '价格': avg_cost,
                    '金额': total_cost,
                    '手续费': 0,
                    '实际成本': total_cost,
                    '备注': holding.get('备注', '初始持仓')
                }])
                transactions_df = pd.concat([transactions_df, holding_record], ignore_index=True)
        
        # 创建持仓记录表
        holdings_df = pd.DataFrame(columns=[
            '资产名称', '持有数量', '平均成本', '总成本', '当前价格', '当前市值', 
            '盈亏金额', '盈亏比例', '价格更新日期', '最后更新日期'
        ])
        
        # 添加现金持仓
        if initial_cash > 0:
            cash_holding = pd.DataFrame([{
                '资产名称': '现金',
                '持有数量': initial_cash,
                '平均成本': 1,
                '总成本': initial_cash,
                '当前价格': 1,
                '当前市值': initial_cash,
                '盈亏金额': 0,
                '盈亏比例': 0,
                '价格更新日期': current_date,
                '最后更新日期': current_date
            }])
            holdings_df = pd.concat([holdings_df, cash_holding], ignore_index=True)
        
        # 添加现有持仓
        if existing_holdings:
            for holding in existing_holdings:
                asset_name = holding['资产名称']
                quantity = holding['持有数量']
                profit_pct = holding.get('盈亏比例', 0)
                
                # 判断使用哪种方式初始化
                if '当前净值' in holding:
                    # 方式1: 使用当前净值和盈亏比例反推平均成本
                    current_price = holding['当前净值']
                    current_value = quantity * current_price
                    
                    # 根据盈亏比例反推总成本
                    # 当前市值 = 总成本 × (1 + 盈亏比例/100)
                    # 总成本 = 当前市值 / (1 + 盈亏比例/100)
                    if profit_pct != 0:
                        total_cost = current_value / (1 + profit_pct / 100)
                    else:
                        total_cost = current_value
                    
                    avg_cost = total_cost / quantity
                    profit_amount = current_value - total_cost
                    
                elif '平均成本' in holding:
                    # 方式2: 使用平均成本
                    avg_cost = holding['平均成本']
                    total_cost = quantity * avg_cost
                    
                    # 如果提供了盈亏比例,计算当前价格和市值
                    if profit_pct != 0:
                        current_value = total_cost * (1 + profit_pct / 100)
                        current_price = current_value / quantity
                        profit_amount = current_value - total_cost
                    else:
                        current_price = avg_cost
                        current_value = total_cost
                        profit_amount = 0
                else:
                    raise ValueError(f"资产 {asset_name} 必须提供 '当前净值' 或 '平均成本'")
                
                new_holding = pd.DataFrame([{
                    '资产名称': asset_name,
                    '持有数量': quantity,
                    '平均成本': avg_cost,
                    '总成本': total_cost,
                    '当前价格': current_price,
                    '当前市值': current_value,
                    '盈亏金额': profit_amount,
                    '盈亏比例': profit_pct,
                    '价格更新日期': current_date if profit_pct != 0 else '',
                    '最后更新日期': current_date
                }])
                holdings_df = pd.concat([holdings_df, new_holding], ignore_index=True)
        
        # 写入Excel
        with pd.ExcelWriter(self.filename, engine='openpyxl') as writer:
            transactions_df.to_excel(writer, sheet_name=self.transactions_sheet, index=False)
            holdings_df.to_excel(writer, sheet_name=self.holdings_sheet, index=False)
        
        print(f"✓ 投资组合已初始化")
        print(f"✓ 初始现金: ¥{initial_cash:,.2f}")
        if existing_holdings:
            print(f"✓ 导入持仓: {len(existing_holdings)} 项资产")
        print(f"✓ 文件已创建: {self.filename}")
        
        # 显示初始持仓概览(含盈亏)
        if not holdings_df.empty:
            self._print_profit_summary(holdings_df)
        
    def add_transaction(self, operation_type, asset_name, quantity, price, fee=0, note=''):
        """
        添加交易记录
        
        参数:
            operation_type: 操作类型 ('买入', '卖出', '现金注入', '现金提取', '分红')
            asset_name: 资产名称
            quantity: 数量
            price: 价格(对于现金注入/提取,这是金额)
            fee: 手续费(默认为0)
            note: 备注
        """
        try:
            # 读取现有数据
            transactions_df = pd.read_excel(self.filename, sheet_name=self.transactions_sheet)
            holdings_df = pd.read_excel(self.filename, sheet_name=self.holdings_sheet)
            
            current_date = datetime.now().strftime('%Y-%m-%d')
            
            # 计算交易金额和实际成本
            if operation_type in ['现金注入', '现金提取']:
                amount = price  # 对于现金操作,price就是金额
                actual_cost = amount
            else:
                amount = quantity * price
                # 买入时:实际成本 = 金额 + 手续费
                # 卖出时:实际收入 = 金额 - 手续费
                if operation_type == '买入':
                    actual_cost = amount + fee
                elif operation_type == '卖出':
                    actual_cost = amount - fee
                else:  # 分红等其他操作
                    actual_cost = amount - fee
            
            # 添加新交易记录
            new_transaction = pd.DataFrame([{
                '日期': current_date,
                '操作类型': operation_type,
                '资产名称': asset_name,
                '数量': quantity,
                '价格': price,
                '金额': amount,
                '手续费': fee,
                '实际成本': actual_cost,
                '备注': note
            }])
            
            transactions_df = pd.concat([transactions_df, new_transaction], ignore_index=True)
            
            # 更新持仓
            holdings_df = self._update_holdings(holdings_df, operation_type, asset_name, 
                                               quantity, price, amount, actual_cost, current_date)
            
            # 保存到Excel
            with pd.ExcelWriter(self.filename, engine='openpyxl', mode='a', 
                              if_sheet_exists='replace') as writer:
                transactions_df.to_excel(writer, sheet_name=self.transactions_sheet, index=False)
                holdings_df.to_excel(writer, sheet_name=self.holdings_sheet, index=False)
            
            print(f"✓ 交易记录已添加: {operation_type} {asset_name}")
            self._print_summary(holdings_df)
            
            return True
            
        except Exception as e:
            print(f"✗ 添加交易失败: {str(e)}")
            return False
    
    def _update_holdings(self, holdings_df, operation_type, asset_name, 
                        quantity, price, amount, actual_cost, current_date):
        """更新持仓信息(考虑手续费)"""
        
        # 查找资产是否已存在
        asset_exists = asset_name in holdings_df['资产名称'].values
        
        if operation_type == '买入':
            # 扣除现金(包含手续费)
            cash_idx = holdings_df[holdings_df['资产名称'] == '现金'].index
            if len(cash_idx) > 0:
                holdings_df.loc[cash_idx[0], '持有数量'] -= actual_cost
                holdings_df.loc[cash_idx[0], '总成本'] -= actual_cost
                holdings_df.loc[cash_idx[0], '最后更新日期'] = current_date
            
            # 增加资产(成本包含手续费)
            if asset_exists:
                idx = holdings_df[holdings_df['资产名称'] == asset_name].index[0]
                old_quantity = holdings_df.loc[idx, '持有数量']
                old_cost = holdings_df.loc[idx, '总成本']
                new_quantity = old_quantity + quantity
                new_cost = old_cost + actual_cost
                holdings_df.loc[idx, '持有数量'] = new_quantity
                holdings_df.loc[idx, '平均成本'] = new_cost / new_quantity
                holdings_df.loc[idx, '总成本'] = new_cost
                holdings_df.loc[idx, '最后更新日期'] = current_date
            else:
                new_holding = pd.DataFrame([{
                    '资产名称': asset_name,
                    '持有数量': quantity,
                    '平均成本': actual_cost / quantity,  # 包含手续费的平均成本
                    '总成本': actual_cost,
                    '最后更新日期': current_date
                }])
                holdings_df = pd.concat([holdings_df, new_holding], ignore_index=True)
        
        elif operation_type == '卖出':
            if asset_exists:
                idx = holdings_df[holdings_df['资产名称'] == asset_name].index[0]
                cost_per_unit = holdings_df.loc[idx, '平均成本']
                holdings_df.loc[idx, '持有数量'] -= quantity
                holdings_df.loc[idx, '总成本'] -= cost_per_unit * quantity
                holdings_df.loc[idx, '最后更新日期'] = current_date
                
                # 如果卖完了,删除该行
                if holdings_df.loc[idx, '持有数量'] <= 0:
                    holdings_df = holdings_df.drop(idx).reset_index(drop=True)
            
            # 增加现金(扣除手续费后)
            cash_idx = holdings_df[holdings_df['资产名称'] == '现金'].index
            if len(cash_idx) > 0:
                holdings_df.loc[cash_idx[0], '持有数量'] += actual_cost
                holdings_df.loc[cash_idx[0], '总成本'] += actual_cost
                holdings_df.loc[cash_idx[0], '最后更新日期'] = current_date
        
        elif operation_type == '现金注入':
            cash_idx = holdings_df[holdings_df['资产名称'] == '现金'].index
            if len(cash_idx) > 0:
                holdings_df.loc[cash_idx[0], '持有数量'] += amount
                holdings_df.loc[cash_idx[0], '总成本'] += amount
                holdings_df.loc[cash_idx[0], '最后更新日期'] = current_date
        
        elif operation_type == '现金提取':
            cash_idx = holdings_df[holdings_df['资产名称'] == '现金'].index
            if len(cash_idx) > 0:
                holdings_df.loc[cash_idx[0], '持有数量'] -= amount
                holdings_df.loc[cash_idx[0], '总成本'] -= amount
                holdings_df.loc[cash_idx[0], '最后更新日期'] = current_date
        
        elif operation_type == '分红':
            # 分红增加现金
            cash_idx = holdings_df[holdings_df['资产名称'] == '现金'].index
            if len(cash_idx) > 0:
                holdings_df.loc[cash_idx[0], '持有数量'] += amount
                holdings_df.loc[cash_idx[0], '总成本'] += amount
                holdings_df.loc[cash_idx[0], '最后更新日期'] = current_date
        
        return holdings_df
    
    def update_current_prices(self, price_dict):
        """
        更新资产当前价格并计算盈亏
        
        参数:
            price_dict: 价格字典,格式为 {'资产名称': 当前价格}
                       例如: {'贵州茅台': 1680.00, '腾讯控股': 395.00}
        """
        try:
            holdings_df = pd.read_excel(self.filename, sheet_name=self.holdings_sheet)
            current_date = datetime.now().strftime('%Y-%m-%d')
            
            # 添加当前价格、市值、盈亏等列(如果不存在)
            if '当前价格' not in holdings_df.columns:
                holdings_df['当前价格'] = 0
            if '当前市值' not in holdings_df.columns:
                holdings_df['当前市值'] = 0
            if '盈亏金额' not in holdings_df.columns:
                holdings_df['盈亏金额'] = 0
            if '盈亏比例' not in holdings_df.columns:
                holdings_df['盈亏比例'] = 0
            if '价格更新日期' not in holdings_df.columns:
                holdings_df['价格更新日期'] = ''
            
            # 更新价格
            for asset_name, current_price in price_dict.items():
                if asset_name in holdings_df['资产名称'].values:
                    idx = holdings_df[holdings_df['资产名称'] == asset_name].index[0]
                    quantity = holdings_df.loc[idx, '持有数量']
                    avg_cost = holdings_df.loc[idx, '平均成本']
                    total_cost = holdings_df.loc[idx, '总成本']
                    
                    # 计算市值和盈亏
                    current_value = quantity * current_price
                    profit_loss = current_value - total_cost
                    profit_loss_pct = (profit_loss / total_cost * 100) if total_cost > 0 else 0
                    
                    # 更新数据
                    holdings_df.loc[idx, '当前价格'] = current_price
                    holdings_df.loc[idx, '当前市值'] = current_value
                    holdings_df.loc[idx, '盈亏金额'] = profit_loss
                    holdings_df.loc[idx, '盈亏比例'] = profit_loss_pct
                    holdings_df.loc[idx, '价格更新日期'] = current_date
            
            # 现金的当前价格设为1
            cash_idx = holdings_df[holdings_df['资产名称'] == '现金'].index
            if len(cash_idx) > 0:
                holdings_df.loc[cash_idx[0], '当前价格'] = 1
                holdings_df.loc[cash_idx[0], '当前市值'] = holdings_df.loc[cash_idx[0], '持有数量']
                holdings_df.loc[cash_idx[0], '盈亏金额'] = 0
                holdings_df.loc[cash_idx[0], '盈亏比例'] = 0
                holdings_df.loc[cash_idx[0], '价格更新日期'] = current_date
            
            # 保存
            with pd.ExcelWriter(self.filename, engine='openpyxl', mode='a', 
                              if_sheet_exists='replace') as writer:
                transactions_df = pd.read_excel(self.filename, sheet_name=self.transactions_sheet)
                transactions_df.to_excel(writer, sheet_name=self.transactions_sheet, index=False)
                holdings_df.to_excel(writer, sheet_name=self.holdings_sheet, index=False)
            
            print(f"✓ 价格已更新({len(price_dict)}项资产)")
            print(f"✓ 更新日期: {current_date}")
            self._print_profit_summary(holdings_df)
            
            return True
            
        except Exception as e:
            print(f"✗ 更新价格失败: {str(e)}")
            return False
    
    def _print_profit_summary(self, holdings_df):
        """打印盈亏摘要"""
        print("\n当前持仓与盈亏:")
        print("-" * 95)
        print(f"{'资产名称':15s} {'数量':>10s} {'成本价':>10s} {'现价':>10s} {'市值':>12s} {'盈亏':>12s} {'盈亏%':>8s}")
        print("-" * 95)
        
        total_cost = 0
        total_value = 0
        
        for _, row in holdings_df.iterrows():
            asset_name = row['资产名称']
            quantity = row['持有数量']
            avg_cost = row['平均成本']
            cost = row['总成本']
            
            # 获取当前价格和市值
            current_price = row.get('当前价格', avg_cost)
            current_value = row.get('当前市值', cost)
            profit_loss = row.get('盈亏金额', 0)
            profit_loss_pct = row.get('盈亏比例', 0)
            
            total_cost += cost
            total_value += current_value
            
            # 盈亏显示颜色标记(用+/-表示)
            pnl_str = f"{profit_loss:+,.2f}" if profit_loss != 0 else "0.00"
            pct_str = f"{profit_loss_pct:+.2f}%" if profit_loss != 0 else "0.00%"
            
            print(f"{asset_name:15s} {quantity:>10.2f} {avg_cost:>10.2f} {current_price:>10.2f} "
                  f"¥{current_value:>11,.2f} ¥{pnl_str:>11s} {pct_str:>8s}")
        
        print("-" * 95)
        total_pnl = total_value - total_cost
        total_pnl_pct = (total_pnl / total_cost * 100) if total_cost > 0 else 0
        
        pnl_str = f"{total_pnl:+,.2f}"
        pct_str = f"{total_pnl_pct:+.2f}%"
        
        print(f"{'总计':15s} {'':10s} {'':10s} {'':10s} "
              f"¥{total_value:>11,.2f} ¥{pnl_str:>11s} {pct_str:>8s}")
        print(f"\n总成本: ¥{total_cost:,.2f} | 总市值: ¥{total_value:,.2f} | 总盈亏: ¥{pnl_str} ({pct_str})")
        print()
        """打印持仓摘要"""
        print("\n当前持仓概览:")
        print("-" * 60)
        total_value = 0
        for _, row in holdings_df.iterrows():
            value = row['持有数量'] if row['资产名称'] == '现金' else row['总成本']
            total_value += value
            print(f"{row['资产名称']:12s} | 数量: {row['持有数量']:>10.2f} | 成本: ¥{row['总成本']:>12,.2f}")
        print("-" * 60)
        print(f"{'总资产':12s} | {'':14s} | ¥{total_value:>12,.2f}")
        print()
    
    def view_holdings(self, show_profit=False):
        """
        查看当前持仓
        
        参数:
            show_profit: 是否显示盈亏信息(需要先更新价格)
        """
        try:
            holdings_df = pd.read_excel(self.filename, sheet_name=self.holdings_sheet)
            print("\n=== 当前持仓 ===")
            
            if show_profit and '当前价格' in holdings_df.columns:
                self._print_profit_summary(holdings_df)
            else:
                self._print_summary(holdings_df)
        except Exception as e:
            print(f"✗ 读取持仓失败: {str(e)}")
    def _print_summary(self, holdings_df):
        """打印持仓摘要(不含盈亏)"""
        print("\n当前持仓概览:")
        print("-" * 60)
        print(f"{'资产名称':15s} {'持有数量':>12s} {'平均成本':>12s} {'总成本':>15s}")
        print("-" * 60)
        
        total_cost = 0
        for _, row in holdings_df.iterrows():
            asset_name = row['资产名称']
            quantity = row['持有数量']
            avg_cost = row['平均成本']
            cost = row['总成本']
            total_cost += cost
            
            print(f"{asset_name:15s} {quantity:>12.2f} {avg_cost:>12.2f} ¥{cost:>14,.2f}")
        
        print("-" * 60)
        print(f"{'总资产':15s} {'':12s} {'':12s} ¥{total_cost:>14,.2f}")
        print()
    def view_transactions(self, last_n=10):
        """查看最近的交易记录"""
        try:
            transactions_df = pd.read_excel(self.filename, sheet_name=self.transactions_sheet)
            print(f"\n=== 最近{last_n}条交易记录 ===")
            print(transactions_df.tail(last_n).to_string(index=False))
            print()
        except Exception as e:
            print(f"✗ 读取交易记录失败: {str(e)}")


# 使用示例
if __name__ == "__main__":
    # 创建跟踪器实例
    tracker = PortfolioTracker('my_portfolio.xlsx')
    
    # ========== 方式1: 从零开始 ==========
    # tracker.initialize_portfolio(initial_cash=100000)
    
    # ========== 方式2: 导入现有持仓(适合中途开始记录) ==========
    # 准备现有持仓数据 - 使用当前净值和盈亏比例(推荐方式)
    # my_existing_holdings = [
    #     # 基金类资产:直接用当前净值和盈亏比例
    #     {
    #         '资产名称': '华泰柏瑞中证红利低波动ETF链接A', 
    #         '持有数量': 16506.42, 
    #         '当前净值': 1.6237,  # 从APP查到的当前净值
    #         '盈亏比例': -4.32  # 从APP查到的盈亏百分比
    #     },
    #     {
    #         '资产名称': '标普500', 
    #         '持有数量': 1956.76, 
    #         '当前净值': 2.0337,
    #         '盈亏比例': 7.8
    #     },
    #     # 股票类资产:也可以用当前价格和盈亏比例
    #     {
    #         '资产名称': '纳斯达克', 
    #         '持有数量': 523.74, 
    #         '当前净值':5.5949 ,  # 当前股价
    #         '盈亏比例': 6.59
    #     },
        
    #     {
    #         '资产名称': '兴全合宜混合A', 
    #         '持有数量': 3862.11, 
    #         '当前净值':2.1244 ,  # 当前股价
    #         '盈亏比例': 13.95
    #     },
    #     {
    #         '资产名称': '立讯精密', 
    #         '持有数量': 100, 
    #         '当前净值':64.69 ,  # 当前股价
    #         '盈亏比例': -6.86
    #     },
    #     {
    #         '资产名称': '海螺水泥', 
    #         '持有数量': 100, 
    #         '当前净值':23.22 ,  # 当前股价
    #         '盈亏比例': -0.73
    #     },
    # ]
    
    # # 初始化(包含现金和现有持仓)
    # tracker.initialize_portfolio(
    #     initial_cash=16000,  # 当前现金余额
    #     existing_holdings=my_existing_holdings
    # )
    
    # # ========== 之后每周记录新的交易 ==========
    # # 买入股票(带手续费)
    # tracker.add_transaction('买入', '比亚迪', 100, 250.00, fee=5.00, note='新建仓位')
    
    # # 现金注入
    # tracker.add_transaction('现金注入', '现金', 1, 20000, note='月度定投')
    
    # # 卖出部分股票(有手续费和印花税)
    # tracker.add_transaction('卖出', '贵州茅台', 50, 1680.00, fee=42.00, note='止盈')
    
    # # 收到分红
    # tracker.add_transaction('分红', '腾讯控股', 200, 2.5, fee=50.00, note='分红扣税10%')
    
    # # ========== 每周更新价格,查看盈亏 ==========
    # # 更新各资产的当前价格
    # current_prices = {
    #     '贵州茅台': 1680.00,
    #     '腾讯控股': 395.00,
    #     '沪深300ETF': 4.50,
    #     '纳斯达克100ETF': 3.00,
    #     '比亚迪': 260.00
    # }
    # tracker.update_current_prices(current_prices)
    
    # # 查看持仓(显示盈亏)
    # tracker.view_holdings(show_profit=True)
    
    # # 查看交易历史
    # tracker.view_transactions(last_n=10)

## 2025/10/09 -2025/10/12

In [3]:
tracker.add_transaction('卖出', '海螺水泥', 100, 23.28, fee=5, note='基本面不利')

✓ 交易记录已添加: 卖出 海螺水泥

当前持仓概览:
------------------------------------------------------------
资产名称                    持有数量         平均成本             总成本
------------------------------------------------------------
现金                  18323.00         1.00 ¥     18,323.00
华泰柏瑞中证红利低波动ETF链接A     16506.42         1.70 ¥     28,011.57
标普500                1956.76         1.89 ¥      3,691.52
纳斯达克                  523.74         5.25 ¥      2,749.11
兴全合宜混合A              3862.11         1.86 ¥      7,200.23
立讯精密                  100.00        69.45 ¥      6,945.46
------------------------------------------------------------
总资产                                       ¥     66,920.90



True

In [4]:
tracker.add_transaction('卖出', '立讯精密', 100, 62.32, fee=5, note='止损')

✓ 交易记录已添加: 卖出 立讯精密

当前持仓概览:
------------------------------------------------------------
资产名称                    持有数量         平均成本             总成本
------------------------------------------------------------
现金                  24550.00         1.00 ¥     24,550.00
华泰柏瑞中证红利低波动ETF链接A     16506.42         1.70 ¥     28,011.57
标普500                1956.76         1.89 ¥      3,691.52
纳斯达克                  523.74         5.25 ¥      2,749.11
兴全合宜混合A              3862.11         1.86 ¥      7,200.23
------------------------------------------------------------
总资产                                       ¥     66,202.44



True

In [5]:
tracker.add_transaction('卖出', '华泰柏瑞中证红利低波动ETF链接A', 4126.61, 1.6256, fee=33.54, note='止损')

✓ 交易记录已添加: 卖出 华泰柏瑞中证红利低波动ETF链接A

当前持仓概览:
------------------------------------------------------------
资产名称                    持有数量         平均成本             总成本
------------------------------------------------------------
现金                  31224.68         1.00 ¥     31,224.68
华泰柏瑞中证红利低波动ETF链接A     12379.81         1.70 ¥     21,008.67
标普500                1956.76         1.89 ¥      3,691.52
纳斯达克                  523.74         5.25 ¥      2,749.11
兴全合宜混合A              3862.11         1.86 ¥      7,200.23
------------------------------------------------------------
总资产                                       ¥     65,874.21



True